In [1]:
# LangChain
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

# Other modules
from pydantic import BaseModel, Field
import os
import tempfile
import streamlit as st
import pandas as pd
from dotenv import load_dotenv


In [2]:
# Load environment variables
load_dotenv()

True

## Defining our LLM

In [3]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY") # Loading the OpenAI API Key

In [4]:
# Creating the LLM object
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=OPENAI_API_KEY
)

## Setting the PDF data

### Loading the PDF data & transforming into chunks

In [18]:
loader = PyPDFLoader("data/Descripcion materias.pdf")

data = loader.load() # Loading the data

# Creating our splitter object to chunk the data
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " "]
)

chunks = text_splitter.split_documents(data) # Creating the chunks

In [19]:
chunks[0].page_content # ex: One of the chunks created

'UNIVERSIDAD TECNOLÓGICA DE PANAMÁ \n \n \n \n \nSECRETARÍA GENERAL \n \n \n \n \n \n \nFACULTAD DE INGENIERÍA DE SISTEMAS COMPUTACIONALES \n \n \n \n \n \nDESCRIPCIÓN DE CURSO DE LA CARRERA DE  \nLICENCIATURA EN INGENIERÍA DE SISTEMAS Y COMPUTACIÓN \n \n \n \n \nAPROBADO POR EL CONSEJO ACADÉMICO EN REUNIÓN Nº 5/2003 DEL 23 DE JUNIO DE 2003 CON \nMODIFICACIONES EN REUNIÓN EXTRAORDINARIA Nº 10 -2003 DEL 14 DE NOVIEMBRE DE 2003 Y \nCON MODIFICACIÓN EN EL CONSEJO ACADÉMICO EN LA REUNIÓN Nº8 – 2004 (ORDINARIA) DEL \n19 DE NOVIEMBRE DE 2004 Y MODIFICACIÓN EN CONSEJO ACADÉMICO EN REUNIÓN Nº 05 -2007 \nDEL 6 DE JULIO DE 2007. Y MODIFICACIÓN EN SESIÓN ORDINARIA N° 03 -2008 DEL 11 DE JULIO DE \n2008. MODIFICACIÓN EN REUNIÓN N° 03 -2010 (EXTRAORDINARIA) DEL 26 DE MAYO DE 2010 . \nMODIFICACIONES EN EL C.A. EN REUNION No.03 -2014 DEL 9 DE MAYO DE 2014. MODIFICACIÓN \nEN CONSEJO ACADÉMICO EN REUNIÓN ORDINARIA N°07 -2015 DE 7 DE AGOSTO DE 2015.  \nMODIFICACIÓN EN LA SESIÓN ORDINARIA Nº 10 -2015 DE 1

## Create embeddings

In [7]:
def create_embeddings():
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small",
        api_key=OPENAI_API_KEY,
    )
    return embeddings

embedding_function = create_embeddings()

### Creating the vector database

In [ ]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):
    # Create a list for unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]

    # Ensure that only unique ids are stored
    unique_ids = set()
    unique_chunks = []

    for id, chunk in zip(ids, chunks):
        if id not in unique_ids:
            unique_ids.add(id)
            unique_chunks.append(chunk)
    
    # Create a new Chroma database
    vectorstore = Chroma.from_documents(
        documents=unique_chunks,
        ids=list(unique_ids),
        embedding=embedding_function,
        persist_directory=vectorstore_path
    )

    vectorstore.persist()

    return vectorstore

In [20]:
vectorstore = create_vectorstore(
    chunks=chunks,
    embedding_function=embedding_function,
    vectorstore_path="vectorstore_chroma"
)

### Query for relevant data

In [21]:
# Load vectorstore
vectorstore = Chroma(persist_directory="vectorstore_chroma", embedding_function=embedding_function)

In [22]:
# Creating the retriever with the parameter search_type="similarity"
retriever = vectorstore.as_retriever(search_type="similarity") 

# Retrieving the relevant chunks
relevant_chunks = retriever.invoke("Cual es la descripcion de la materia de 'ingenieria de software I'")
relevant_chunks

[Document(metadata={'moddate': '2024-05-10T18:26:23-05:00', 'creator': 'Microsoft® Word 2016', 'source': 'data/Descripcion materias.pdf', 'creationdate': '2024-05-10T18:26:23-05:00', 'title': 'UNIVERSIDAD TECNOLOGICA DE PANAMA', 'total_pages': 19, 'producer': 'Microsoft® Word 2016', 'page': 12, 'author': 'Addys', 'page_label': '13'}, page_content='apreciar los aspectos de hardware. Finalmente, la interacción con robots dotados de sistemas de \nmovilidad y percepción avanzados permitirá apreciar el potencial, complejidad y l imitaciones \nactuales de la tecnología, así como los aspectos más relevantes del software requerido. Se \npersigue que el estudiante desarrolle un proyecto de robótica que incluya el diseño, la \nprogramación y/o construcción de un sistema robótico, con énfasis  en aplicaciones prácticas en \nbase a sistemas inteligentes y en tareas de representación del espacio, construcción de mapas \nsensoriales, auto localización, seguimiento de trayectorias e interacción con o

## Generate responses

In [14]:
# Prompt template
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If the answer is not contained within the context, say "I don't know".

Context: {context}

Answer the question based on the above context: {question}
"""

In [ ]:
# Saving the context
context_text = "\n\n---\n\n".join([chunk.page_content for chunk in chunks])

# Creating the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question="Que materias tengo en mi primer semestre de mi carrera?")

print(prompt)

### Using langchain expression language

In [35]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
)

rag_chain.invoke("Cual es la descripcion de la materia de ingenieria de software I?")

AIMessage(content='En el curso de Ingeniería de Software I, los estudiantes aprenden los fundamentos del proceso de desarrollo iterativo. Se concentran en el análisis de requerimientos, definición de condiciones o capacidades necesarias para los usuarios, ofreciendo soluciones a un problema o en la consecución de un proyecto. También se trabaja en la especificación del comportamiento externo de los sistemas, arquitectura general de los sistemas, y los componentes físicos de los mismos.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 1466, 'total_tokens': 1551, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6b23b3aa8a', 'id': 'chatcmpl-D5YBTuObphhQoO4UzEQKL9QSF8ZJ6', 'ser